### Test First Ray

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Build one ray + spectrum from saved endpoints (ckpc/h) using yt + trident.

- Reads rays_and_recipes_sid<SID>_snap<SNAP>/rays_sid<SID>.csv (absolute ckpc/h)
- Picks the FIRST row (optionally filter by mode="noflip"/"flip")
- Creates a trident ray (start/end in code_length)
- Ensures H I, C II, Si III ion number density fields exist
- Computes integrated columns (sum n_ion * dl); uses native dl if present,
  else geometric fallback from ordered cell centers along the segment
- Synthesizes spectra for ["H I 1216", "C II 1334", "Si III 1206"] with COS-G130M LSF
- Saves figures + a compact HDF5 bundle (meta, spectra, small ray arrays)
"""

import os, json, traceback
import numpy as np
import pandas as pd
import yt
import trident
import h5py
import matplotlib.pyplot as plt

# =======================
# --- YOUR LOCAL PATHS ---
# =======================
CUTOUT_H5 = "../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/cutout_ALLFIELDS_sphere_2p1Rvir_sub563732.hdf5"

SID  = 563732
SNAP = 99

OUTROOT   = f"rays_and_recipes_sid{SID}_snap{SNAP}"
RAYS_CSV  = os.path.join(OUTROOT, f"rays_sid{SID}.csv")
OUT_DIR   = os.path.join(OUTROOT, "ray_and_spectrum")

# Optional filter (set to None to use the very first row)
FILTER_MODE = None        # "noflip" | "flip" | None

# Lines & instrument (C II is 1334)
LINES = ["H I 1216", "C II 1335", "Si III 1206"]
INSTR = "COS-G130M"

# Zoom windows (Å) for the zoom PNG
ZOOMS_A = {
    "Lyα 1215.67": 1215.67,
    "Si III 1206.50": 1206.50,
    "C II 1334.53": 1334.53,
}
ZOOM_HALF_A = 3.0


# -----------------------
# Utility helpers
# -----------------------
def ensure_dir(p):
    os.makedirs(p, exist_ok=True)
    return p

def pick_first_row(rays_csv, mode=None):
    """
    Expect columns:
      p0_X_ckpch_abs ... p0_Z_ckpch_abs, p1_X_ckpch_abs ... p1_Z_ckpch_abs,
      mode, alpha_deg, SubhaloID, rho_kpc, phi_deg, inc_deg, Rvir_kpc
    """
    df = pd.read_csv(rays_csv)
    print(f"[INFO] Loaded rays CSV: {rays_csv}  (rows={len(df)})")
    if mode is not None:
        df = df[df["mode"].str.lower() == str(mode).lower()]
        print(f"[INFO] Filtered by mode='{mode}'  -> remaining rows={len(df)}")
        if df.empty:
            raise RuntimeError(f"No rows after mode filter: {mode}")
    row = df.iloc[0]
    p0 = np.array([row["p0_X_ckpch_abs"], row["p0_Y_ckpch_abs"], row["p0_Z_ckpch_abs"]], float)
    p1 = np.array([row["p1_X_ckpch_abs"], row["p1_Y_ckpch_abs"], row["p1_Z_ckpch_abs"]], float)
    meta = {
        "SubhaloID": int(row.get("SubhaloID", -1)),
        "alpha_deg": float(row.get("alpha_deg", np.nan)),
        "mode":      str(row.get("mode", "unknown")),
        "rho_kpc":   float(row.get("rho_kpc", np.nan)),
        "phi_deg":   float(row.get("phi_deg", np.nan)),
        "inc_deg":   float(row.get("inc_deg", np.nan)),
        "Rvir_kpc":  float(row.get("Rvir_kpc", np.nan)),
    }
    return p0, p1, meta

def add_ions(ds):
    """Add ion fields if not present. Trident won’t clobber existing ones."""
    needed = ["H I", "C II", "Si III"]
    try:
        trident.add_ion_fields(ds, ions=needed)
        print("[INFO] Ion fields added (or already present):", needed)
    except Exception as e:
        print("[WARN] add_ion_fields failed (fields may already exist):", e)

def make_ray(ds, p0_ckpch, p1_ckpch):
    """
    Build a ray with start/end in absolute *code_length* (ckpc/h).
    NOTE: do NOT request ('gas','dl') or ('gas','velocity_los') during build.
    """
    sp = ds.arr(p0_ckpch, "code_length")
    ep = ds.arr(p1_ckpch, "code_length")
    print(f"[RAY] Start (code_length): {sp}")
    print(f"[RAY] End   (code_length): {ep}")

    # Domain sanity
    L = ds.domain_left_edge.to("code_length").value
    R = ds.domain_right_edge.to("code_length").value
    for name, vec in [("start", sp.value), ("end", ep.value)]:
        inside = np.all(vec >= L - 1e-6) and np.all(vec <= R + 1e-6)
        print(f"[RAY] {name} inside domain: {inside}")

    fields = [
        ("gas", "density"),
        ("gas", "temperature"),
        ("gas", "metallicity"),
        ("gas", "H_p0_number_density"),
        ("gas", "C_p1_number_density"),
        ("gas", "Si_p2_number_density"),
        ("gas", "H_number_density"),
        # If needed later: ("gas","velocity_x"),("gas","velocity_y"),("gas","velocity_z")
    ]

    ray = trident.make_simple_ray(
        ds,
        start_position=sp,
        end_position=ep,
        fields=fields,
        ftype="gas"
    )

    ad = ray.all_data()
    nseg = ad[('gas','density')].size
    geom_len_ckpch = np.sqrt(((p1_ckpch - p0_ckpch)**2).sum())
    print(f"[RAY] N segments: {nseg} | geometric length ≈ {geom_len_ckpch:.3f} ckpc/h")
    return ray

def get_dl_cm(ray, p0_ckpch=None, p1_ckpch=None, verbose=True):
    """
    Try to obtain a per-cell path length in cm.
    1) Preferred: use ('gas','dl') if present on the final ray dataset.
    2) Fallback: approximate from ordered cell centers along the ray using p0/p1.
    """
    ad = ray.all_data()
    # Try native fields
    for cand in [( "gas","dl"), ("dl",), ("gas","segment_length"), ("gas","path_length")]:
        try:
            arr = ad[cand].to("cm").value
            if verbose:
                print(f"[DL] Using native field {cand} (min={arr.min():.3e} cm, max={arr.max():.3e} cm)")
            return arr, "native"
        except Exception:
            pass

    # fallback: geometric approximation from ordered centers
    if p0_ckpch is None or p1_ckpch is None:
        raise RuntimeError("No dl-like field and no endpoints provided for approximation.")

    X = np.vstack([
        ad[("gas","x")].to("code_length").value,
        ad[("gas","y")].to("code_length").value,
        ad[("gas","z")].to("code_length").value
    ]).T
    p0 = np.asarray(p0_ckpch, float); p1 = np.asarray(p1_ckpch, float)
    v  = p1 - p0; L = np.linalg.norm(v)
    if L <= 0:
        raise RuntimeError("Zero-length endpoints; cannot approximate dl.")
    u = v / L  # unit LOS in code_length

    s = (X - p0[None,:]) @ u
    order = np.argsort(s)
    s_sorted = s[order]
    ds_code = np.diff(s_sorted)
    if ds_code.size == 0:
        raise RuntimeError("Ray has <2 segments; cannot approximate dl.")
    dl_sorted = np.empty_like(s_sorted)
    dl_sorted[:-1] = ds_code
    dl_sorted[-1]  = ds_code[-1]

    conv = ray.ds.arr(1.0, "code_length").to("cm").value
    dl_cm_sorted = dl_sorted * conv

    inv = np.empty_like(order); inv[order] = np.arange(order.size)
    dl_cm = dl_cm_sorted[inv]
    if verbose:
        print(f"[DL] Approximated from centers (code_length→cm). Σdl≈{dl_cm.sum()/3.086e21:.3f} kpc")
    return dl_cm, "approx"

def _try_get(ad, fld, unit):
    """Robust field grab: returns np.array or None."""
    try:
        return ad[fld].to(unit).value
    except Exception:
        return None

def compute_columns(ray, p0_ckpch, p1_ckpch):
    """
    Integrate columns along the ray: sum(n_ion * dl) in cm^-2.
    Uses native dl if present else geometric approximation.
    """
    ad = ray.all_data()
    dl_cm, dl_kind = get_dl_cm(ray, p0_ckpch, p1_ckpch, verbose=True)

    out = {"_dl_kind": dl_kind, "_sum_dl_kpc": float(dl_cm.sum()/3.08567758e21)}

    def add_if_present(fld, key):
        n = _try_get(ad, fld, "cm**-3")
        if n is not None:
            out[key] = float((n * dl_cm).sum())
        else:
            out[key] = np.nan

    add_if_present(("gas","H_p0_number_density"),  "N_HI_cm2")
    add_if_present(("gas","C_p1_number_density"),  "N_CII_cm2")
    add_if_present(("gas","Si_p2_number_density"), "N_SiIII_cm2")

    print("=== Integrated columns (cm^-2) ===")
    for k in ["N_HI_cm2","N_CII_cm2","N_SiIII_cm2"]:
        v = out[k]
        lg = np.log10(v) if (v is not None and np.isfinite(v) and v > 0) else -np.inf
        print(f"{k:10s}: {v:.6e}   (logN = {lg:.3f})")
    print(f"[DL] kind={dl_kind} | Σdl ≈ {out['_sum_dl_kpc']:.3f} kpc")
    return out

def build_spectrum(ray, lines, instr="COS-G130M"):
    sg = trident.SpectrumGenerator(instr)
    sg.make_spectrum(ray, lines=lines)

    lam = np.array(sg.lambda_field)   # Å
    tau = np.array(sg.tau_field)      # dimensionless
    flux_lsf = np.array(sg.flux_field)
    flux_raw = np.exp(-tau)

    print(f"[SPEC] λ grid: {lam.min():.2f}–{lam.max():.2f} Å  (N={lam.size})  |  instrument={instr}")
    return {
        "raw": {"lambda_A": lam, "flux": flux_raw, "tau": tau},
        "lsf": {"lambda_A": lam, "flux": flux_lsf, "tau": tau},
    }

def save_spectrum_pngs(spec, out_dir, tag="ray"):
    ensure_dir(out_dir)
    lam_r, T_r = spec["raw"]["lambda_A"], spec["raw"]["flux"]
    lam_l, T_l = spec["lsf"]["lambda_A"], spec["lsf"]["flux"]

    # Full spectrum
    fig = plt.figure(figsize=(10, 4))
    ax = fig.add_subplot(111)
    ax.plot(lam_r, T_r, lw=1.1, label="raw")
    ax.plot(lam_l, T_l, lw=1.1, alpha=0.9, label="LSF (COS-G130M)")
    ax.set_ylim(-0.05, 1.05)
    ax.set_xlabel("Wavelength [Å]"); ax.set_ylabel("Transmission")
    ax.set_title("Full spectrum")
    ax.legend(); ax.grid(alpha=0.25)
    plt.tight_layout()
    path = os.path.join(out_dir, f"{tag}_spectrum_full.png")
    plt.savefig(path, dpi=160); plt.close(fig)
    print("[SAVED]", path)

    # Zooms
    fig, axes = plt.subplots(1, len(ZOOMS_A), figsize=(4.0*len(ZOOMS_A), 3), sharey=True)
    if len(ZOOMS_A) == 1:
        axes = [axes]
    for ax, (name, lam0) in zip(axes, ZOOMS_A.items()):
        m  = (lam_r >= lam0 - ZOOM_HALF_A) & (lam_r <= lam0 + ZOOM_HALF_A)
        ml = (lam_l >= lam0 - ZOOM_HALF_A) & (lam_l <= lam0 + ZOOM_HALF_A)
        ax.plot(lam_r[m], T_r[m], lw=1.0, label="raw")
        ax.plot(lam_l[ml], T_l[ml], lw=1.0, alpha=0.9, label="LSF")
        ax.axvline(lam0, color="k", ls="--", lw=0.8, alpha=0.6)
        ax.set_xlim(lam0 - ZOOM_HALF_A, lam0 + ZOOM_HALF_A)
        ax.set_title(name); ax.set_xlabel("λ [Å]")
        ax.grid(alpha=0.25)
    axes[0].set_ylabel("Transmission"); axes[0].legend(frameon=False)
    plt.tight_layout()
    path = os.path.join(out_dir, f"{tag}_spectrum_zooms.png")
    plt.savefig(path, dpi=160); plt.close(fig)
    print("[SAVED]", path)

def save_bundle_hdf5(out_path, meta, ray, spec, cols):
    ensure_dir(os.path.dirname(out_path))
    with h5py.File(out_path, "w") as f:
        # meta
        gmeta = f.create_group("meta")
        for k, v in meta.items():
            try: gmeta.attrs[k] = v
            except TypeError: gmeta.attrs[k] = json.dumps(v)

        # columns
        gcols = f.create_group("columns")
        for k, v in cols.items():
            gcols.attrs[k] = v if isinstance(v, (int,float)) else json.dumps(v)

        # spectra
        gs = f.create_group("spectrum")
        for tag in ("raw","lsf"):
            gt = gs.create_group(tag)
            gt.create_dataset("lambda_A", data=np.asarray(spec[tag]["lambda_A"]))
            gt.create_dataset("flux",     data=np.asarray(spec[tag]["flux"]))
            gt.create_dataset("tau",      data=np.asarray(spec[tag]["tau"]))

        # small ray dump
        ad = ray.all_data()
        gray = f.create_group("ray")
        for fld, name, unit in [
            (("gas","H_p0_number_density"), "nHI_cm3",    "cm**-3"),
            (("gas","C_p1_number_density"), "nCII_cm3",   "cm**-3"),
            (("gas","Si_p2_number_density"),"nSiIII_cm3", "cm**-3"),
            (("gas","density"),              "rho_cgs",    "g/cm**3"),
            (("gas","temperature"),          "T_K",        "K"),
            (("gas","x"),                    "x_code",     "code_length"),
            (("gas","y"),                    "y_code",     "code_length"),
            (("gas","z"),                    "z_code",     "code_length"),
        ]:
            try:
                arr = ad[fld].to(unit).value
                gray.create_dataset(name, data=np.asarray(arr))
            except Exception:
                pass
        # also store the dl used for columns:
        try:
            dl_cm, kind = get_dl_cm(ray, meta["start_ckpch"], meta["end_ckpch"], verbose=False)
            gray.create_dataset("dl_cm", data=np.asarray(dl_cm))
            gray.attrs["dl_kind"] = kind
        except Exception:
            pass
    print("[SAVED]", out_path)


# -----------------------
# Main
# -----------------------
def main():
    ensure_dir(OUT_DIR)
    print("[INFO] Loading dataset:", CUTOUT_H5)
    ds = yt.load(CUTOUT_H5)
    print(ds)

    add_ions(ds)

    p0, p1, meta = pick_first_row(RAYS_CSV, mode=FILTER_MODE)
    print(f"[INFO] Picked sightline: sid={meta['SubhaloID']}  mode={meta['mode']}  alpha={meta['alpha_deg']} deg")
    print(f"[INFO] p0={p0}  p1={p1}   (units: ckpc/h)")

    ray = make_ray(ds, p0, p1)
    cols = compute_columns(ray, p0, p1)
    spec = build_spectrum(ray, LINES, instr=INSTR)

    # Outputs
    tag = f"sid{meta['SubhaloID']}_{meta['mode']}_alpha{int(meta['alpha_deg']) if np.isfinite(meta['alpha_deg']) else 'NA'}"
    save_spectrum_pngs(spec, OUT_DIR, tag=tag)

    bundle_path = os.path.join(OUT_DIR, f"{tag}_bundle.h5")
    meta_save = dict(meta)
    meta_save.update(dict(lines=LINES, instrument=INSTR,
                          start_ckpch=p0.tolist(), end_ckpch=p1.tolist()))
    save_bundle_hdf5(bundle_path, meta_save, ray, spec, cols)

    # Tiny CSV summary
    sum_csv = os.path.join(OUT_DIR, f"{tag}_summary.csv")
    row = dict(meta_save); row.update(cols)
    pd.DataFrame([row]).to_csv(sum_csv, index=False)
    print("[SAVED]", sum_csv)

    print("\n[DONE] One ray + spectra created.")
    print("      OUT_DIR =", OUT_DIR)

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print("[FATAL]", e)
        traceback.print_exc()
        raise

yt : [INFO     ] 2025-10-15 20:51:48,894 Calculating time from 1.000e+00 to be 4.356e+17 seconds
yt : [INFO     ] 2025-10-15 20:51:48,914 Parameters: current_time              = 4.355810528213311e+17 s
yt : [INFO     ] 2025-10-15 20:51:48,915 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2025-10-15 20:51:48,915 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-10-15 20:51:48,916 Parameters: domain_right_edge         = [35000. 35000. 35000.]
yt : [INFO     ] 2025-10-15 20:51:48,917 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2025-10-15 20:51:48,918 Parameters: current_redshift          = 2.220446049250313e-16
yt : [INFO     ] 2025-10-15 20:51:48,918 Parameters: omega_lambda              = 0.6911
yt : [INFO     ] 2025-10-15 20:51:48,919 Parameters: omega_matter              = 0.3089
yt : [INFO     ] 2025-10-15 20:51:48,919 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2025-10-15 20:51:48,920 Parameters: hubble_con

[INFO] Loading dataset: ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/cutout_ALLFIELDS_sphere_2p1Rvir_sub563732.hdf5
cutout_ALLFIELDS_sphere_2p1Rvir_sub563732


Loading particle index: 100%|██████████| 13/13 [00:00<00:00, 9993.76it/s]
yt : [WARNING  ] 2025-10-15 20:51:49,149 Field ('gas', 'H_p0_number_density') already exists. Not clobbering.
yt : [WARNING  ] 2025-10-15 20:51:49,149 Field ('gas', 'H_p0_density') already exists. Not clobbering.
yt : [INFO     ] 2025-10-15 20:51:49,161 Getting segment at z = 2.220446049250313e-16: [0.66584318 0.51218094 0.24018352] unitary to [0.66763032 0.51957756 0.25220259] unitary.
yt : [INFO     ] 2025-10-15 20:51:49,162 Getting subsegment: [0.66584318 0.51218094 0.24018352] unitary to [0.66763032 0.51957756 0.25220259] unitary.


[INFO] Ion fields added (or already present): ['H I', 'C II', 'Si III']
[INFO] Loaded rays CSV: rays_and_recipes_sid563732_snap99/rays_sid563732.csv  (rows=720)
[INFO] Picked sightline: sid=563732  mode=noflip  alpha=0.0 deg
[INFO] p0=[23304.51112799 17926.33292137  8406.42318879]  p1=[23367.06130566 18185.21468047  8827.09071356]   (units: ckpc/h)
[RAY] Start (code_length): [23304.51112799 17926.33292137  8406.42318879] code_length
[RAY] End   (code_length): [23367.06130566 18185.21468047  8827.09071356] code_length
[RAY] start inside domain: True
[RAY] end inside domain: True


yt : [INFO     ] 2025-10-15 20:51:49,893 Saving field data to yt dataset: ray.h5.
yt : [INFO     ] 2025-10-15 20:51:49,905 Parameters: current_time              = 4.355810528213311e+17 s
yt : [INFO     ] 2025-10-15 20:51:49,905 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2025-10-15 20:51:49,905 Parameters: domain_left_edge          = [0. 0. 0.] code_length
yt : [INFO     ] 2025-10-15 20:51:49,905 Parameters: domain_right_edge         = [35000. 35000. 35000.] code_length
yt : [INFO     ] 2025-10-15 20:51:49,906 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2025-10-15 20:51:49,906 Parameters: current_redshift          = 2.220446049250313e-16
yt : [INFO     ] 2025-10-15 20:51:49,906 Parameters: omega_lambda              = 0.6911
yt : [INFO     ] 2025-10-15 20:51:49,906 Parameters: omega_matter              = 0.3089
yt : [INFO     ] 2025-10-15 20:51:49,907 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2025-10-15 20:51:49,907 Parameters: h

[RAY] N segments: 329 | geometric length ≈ 497.889 ckpc/h
[DL] Using native field ('gas', 'dl') (min=1.367e+13 cm, max=5.218e+22 cm)
=== Integrated columns (cm^-2) ===
N_HI_cm2  : 1.120879e+13   (logN = 13.050)
N_CII_cm2 : 1.108477e+07   (logN = 7.045)
N_SiIII_cm2: 1.354366e+06   (logN = 6.132)
[DL] kind=native | Σdl ≈ 767.272 kpc


Adding line - Si III 1206 [1206.500000 A]: 100%|██████████| 329/329 [00:00<00:00, 26686.90it/s]
yt : [INFO     ] 2025-10-15 20:51:50,057 Not adding continuum Ly C: insufficient column density or out of range


[SPEC] λ grid: 1150.00–1450.00 Å  (N=30001)  |  instrument=COS-G130M
[SAVED] rays_and_recipes_sid563732_snap99/ray_and_spectrum/sid563732_noflip_alpha0_spectrum_full.png
[SAVED] rays_and_recipes_sid563732_snap99/ray_and_spectrum/sid563732_noflip_alpha0_spectrum_zooms.png
[SAVED] rays_and_recipes_sid563732_snap99/ray_and_spectrum/sid563732_noflip_alpha0_bundle.h5
[SAVED] rays_and_recipes_sid563732_snap99/ray_and_spectrum/sid563732_noflip_alpha0_summary.csv

[DONE] One ray + spectra created.
      OUT_DIR = rays_and_recipes_sid563732_snap99/ray_and_spectrum


### Test all rays

In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Batch: build rays + spectra for *all* rows in rays_sid<SID>.csv using yt + trident.

For each row:
  • Build a trident ray from absolute endpoints (ckpc/h == yt "code_length")
  • Ensure ion fields exist: H I, C II, Si III
  • Integrate columns: sum(n_ion * dl) using native dl (if present) or geometric fallback
  • Generate COS-G130M spectra (raw + LSF)
  • Save per-ray PNGs + compact HDF5 bundle (+ minimal per-ray CSV summary)

All outputs are saved under:
  /Users/wavefunction/ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732
into a subfolder: rays_and_spectra_sid<SID>_snap<SNAP>/

Requires: yt, trident, numpy, pandas, h5py, matplotlib
"""

import os, json, traceback
import numpy as np
import pandas as pd
import yt
import trident
import h5py
import matplotlib.pyplot as plt

# =======================
# --- YOUR LOCAL PATHS ---
# =======================
CUTOUT_H5 = "../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/cutout_ALLFIELDS_sphere_2p1Rvir_sub563732.hdf5"

SID  = 563732
SNAP = 99

# The CSV you already generated earlier (don’t move it unless you want to)
RAYS_CSV = f"rays_and_recipes_sid{SID}_snap{SNAP}/rays_sid{SID}.csv"

# === All outputs will be saved inside this base directory ===
OUTPUT_BASE = "../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732"
JOB_ROOT    = os.path.join(OUTPUT_BASE, f"rays_and_spectra_sid{SID}_snap{SNAP}")
RAYS_DIR    = os.path.join(JOB_ROOT, "rays")      # per-ray subfolders live here
LOGS_DIR    = os.path.join(JOB_ROOT, "logs")      # errors etc.
MASTER_CSV  = os.path.join(JOB_ROOT, "summary_all_rays.csv")

# Optional filter (set to None to run *all* rows)
FILTER_MODE = None        # "noflip" | "flip" | None

# Lines & instrument (C II is 1334)
LINES = ["H I 1216", "C II 1335", "Si III 1206"]
INSTR = "COS-G130M"

# Zoom windows (Å) for the zoom PNG
ZOOMS_A = {
    "Lyα 1215.67": 1215.67,
    "Si III 1206.50": 1206.50,
    "C II 1334.53": 1334.53,
}
ZOOM_HALF_A = 3.0


# -----------------------
# Utility helpers
# -----------------------
def ensure_dir(p):
    os.makedirs(p, exist_ok=True)
    return p

def add_ions(ds):
    """Add ion fields if not present. Trident won’t clobber existing ones."""
    needed = ["H I", "C II", "Si III"]
    try:
        trident.add_ion_fields(ds, ions=needed)
        print("[INFO] Ion fields added (or already present):", needed)
    except Exception as e:
        print("[WARN] add_ion_fields failed (fields may already exist):", e)

def make_ray(ds, p0_ckpch, p1_ckpch):
    """
    Build a ray with start/end in absolute *code_length* (ckpc/h).
    NOTE: do NOT request ('gas','dl') or ('gas','velocity_los') during build.
    """
    sp = ds.arr(p0_ckpch, "code_length")
    ep = ds.arr(p1_ckpch, "code_length")
    print(f"[RAY] Start (code_length): {sp}")
    print(f"[RAY] End   (code_length): {ep}")

    # Domain sanity
    L = ds.domain_left_edge.to("code_length").value
    R = ds.domain_right_edge.to("code_length").value
    for name, vec in [("start", sp.value), ("end", ep.value)]:
        inside = np.all(vec >= L - 1e-6) and np.all(vec <= R + 1e-6)
        print(f"[RAY] {name} inside domain: {inside}")

    fields = [
        ("gas", "density"),
        ("gas", "temperature"),
        ("gas", "metallicity"),
        ("gas", "H_p0_number_density"),
        ("gas", "C_p1_number_density"),
        ("gas", "Si_p2_number_density"),
        ("gas", "H_number_density"),
    ]

    ray = trident.make_simple_ray(
        ds,
        start_position=sp,
        end_position=ep,
        fields=fields,
        ftype="gas"
    )

    ad = ray.all_data()
    nseg = ad[('gas','density')].size
    geom_len_ckpch = np.sqrt(((p1_ckpch - p0_ckpch)**2).sum())
    print(f"[RAY] N segments: {nseg} | geometric length ≈ {geom_len_ckpch:.3f} ckpc/h")
    return ray

def get_dl_cm(ray, p0_ckpch=None, p1_ckpch=None, verbose=True):
    """
    Try to obtain a per-cell path length in cm.
    1) Preferred: use ('gas','dl') if present on the final ray dataset.
    2) Fallback: approximate from ordered cell centers along the ray using p0/p1.
    """
    ad = ray.all_data()
    # Try native fields
    for cand in [( "gas","dl"), ("dl",), ("gas","segment_length"), ("gas","path_length")]:
        try:
            arr = ad[cand].to("cm").value
            if verbose:
                print(f"[DL] Using native field {cand} (min={arr.min():.3e} cm, max={arr.max():.3e} cm)")
            return arr, "native"
        except Exception:
            pass

    # fallback: geometric approximation from ordered centers
    if p0_ckpch is None or p1_ckpch is None:
        raise RuntimeError("No dl-like field and no endpoints provided for approximation.")

    X = np.vstack([
        ad[("gas","x")].to("code_length").value,
        ad[("gas","y")].to("code_length").value,
        ad[("gas","z")].to("code_length").value
    ]).T
    p0 = np.asarray(p0_ckpch, float); p1 = np.asarray(p1_ckpch, float)
    v  = p1 - p0; L = np.linalg.norm(v)
    if L <= 0:
        raise RuntimeError("Zero-length endpoints; cannot approximate dl.")
    u = v / L  # unit LOS in code_length

    s = (X - p0[None,:]) @ u
    order = np.argsort(s)
    s_sorted = s[order]
    ds_code = np.diff(s_sorted)
    if ds_code.size == 0:
        raise RuntimeError("Ray has <2 segments; cannot approximate dl.")
    dl_sorted = np.empty_like(s_sorted)
    dl_sorted[:-1] = ds_code
    dl_sorted[-1]  = ds_code[-1]

    conv = ray.ds.arr(1.0, "code_length").to("cm").value
    dl_cm_sorted = dl_sorted * conv

    inv = np.empty_like(order); inv[order] = np.arange(order.size)
    dl_cm = dl_cm_sorted[inv]
    if verbose:
        print(f"[DL] Approximated from centers (code_length→cm). Σdl≈{dl_cm.sum()/3.086e21:.3f} kpc")
    return dl_cm, "approx"

def _try_get(ad, fld, unit):
    """Robust field grab: returns np.array or None."""
    try:
        return ad[fld].to(unit).value
    except Exception:
        return None

def compute_columns(ray, p0_ckpch, p1_ckpch):
    """
    Integrate columns along the ray: sum(n_ion * dl) in cm^-2.
    Uses native dl if present else geometric approximation.
    """
    ad = ray.all_data()
    dl_cm, dl_kind = get_dl_cm(ray, p0_ckpch, p1_ckpch, verbose=True)

    out = {"_dl_kind": dl_kind, "_sum_dl_kpc": float(dl_cm.sum()/3.08567758e21)}

    def add_if_present(fld, key):
        n = _try_get(ad, fld, "cm**-3")
        if n is not None:
            out[key] = float((n * dl_cm).sum())
        else:
            out[key] = np.nan

    add_if_present(("gas","H_p0_number_density"),  "N_HI_cm2")
    add_if_present(("gas","C_p1_number_density"),  "N_CII_cm2")
    add_if_present(("gas","Si_p2_number_density"), "N_SiIII_cm2")

    print("=== Integrated columns (cm^-2) ===")
    for k in ["N_HI_cm2","N_CII_cm2","N_SiIII_cm2"]:
        v = out[k]
        lg = np.log10(v) if (v is not None and np.isfinite(v) and v > 0) else -np.inf
        print(f"{k:10s}: {v:.6e}   (logN = {lg:.3f})")
    print(f"[DL] kind={dl_kind} | Σdl ≈ {out['_sum_dl_kpc']:.3f} kpc")
    return out

def build_spectrum(ray, lines, instr="COS-G130M"):
    sg = trident.SpectrumGenerator(instr)
    sg.make_spectrum(ray, lines=lines)

    lam = np.array(sg.lambda_field)   # Å
    tau = np.array(sg.tau_field)      # dimensionless
    flux_lsf = np.array(sg.flux_field)
    flux_raw = np.exp(-tau)

    print(f"[SPEC] λ grid: {lam.min():.2f}–{lam.max():.2f} Å  (N={lam.size})  |  instrument={instr}")
    return {
        "raw": {"lambda_A": lam, "flux": flux_raw, "tau": tau},
        "lsf": {"lambda_A": lam, "flux": flux_lsf, "tau": tau},
    }

def save_spectrum_pngs(spec, out_dir, tag="ray"):
    ensure_dir(out_dir)
    lam_r, T_r = spec["raw"]["lambda_A"], spec["raw"]["flux"]
    lam_l, T_l = spec["lsf"]["lambda_A"], spec["lsf"]["flux"]

    # Full spectrum
    fig = plt.figure(figsize=(10, 4))
    ax = fig.add_subplot(111)
    ax.plot(lam_r, T_r, lw=1.1, label="raw")
    ax.plot(lam_l, T_l, lw=1.1, alpha=0.9, label="LSF (COS-G130M)")
    ax.set_ylim(-0.05, 1.05)
    ax.set_xlabel("Wavelength [Å]"); ax.set_ylabel("Transmission")
    ax.set_title("Full spectrum")
    ax.legend(); ax.grid(alpha=0.25)
    plt.tight_layout()
    path = os.path.join(out_dir, f"{tag}_spectrum_full.png")
    plt.savefig(path, dpi=160); plt.close(fig)
    print("[SAVED]", path)

    # Zooms
    fig, axes = plt.subplots(1, len(ZOOMS_A), figsize=(4.0*len(ZOOMS_A), 3), sharey=True)
    if len(ZOOMS_A) == 1:
        axes = [axes]
    for ax, (name, lam0) in zip(axes, ZOOMS_A.items()):
        m  = (lam_r >= lam0 - ZOOM_HALF_A) & (lam_r <= lam0 + ZOOM_HALF_A)
        ml = (lam_l >= lam0 - ZOOM_HALF_A) & (lam_l <= lam0 + ZOOM_HALF_A)
        ax.plot(lam_r[m], T_r[m], lw=1.0, label="raw")
        ax.plot(lam_l[ml], T_l[ml], lw=1.0, alpha=0.9, label="LSF")
        ax.axvline(lam0, color="k", ls="--", lw=0.8, alpha=0.6)
        ax.set_xlim(lam0 - ZOOM_HALF_A, lam0 + ZOOM_HALF_A)
        ax.set_title(name); ax.set_xlabel("λ [Å]")
        ax.grid(alpha=0.25)
    axes[0].set_ylabel("Transmission"); axes[0].legend(frameon=False)
    plt.tight_layout()
    path = os.path.join(out_dir, f"{tag}_spectrum_zooms.png")
    plt.savefig(path, dpi=160); plt.close(fig)
    print("[SAVED]", path)

def save_bundle_hdf5(out_path, meta, ray, spec, cols):
    ensure_dir(os.path.dirname(out_path))
    with h5py.File(out_path, "w") as f:
        # meta
        gmeta = f.create_group("meta")
        for k, v in meta.items():
            try: gmeta.attrs[k] = v
            except TypeError: gmeta.attrs[k] = json.dumps(v)

        # columns
        gcols = f.create_group("columns")
        for k, v in cols.items():
            gcols.attrs[k] = v if isinstance(v, (int,float)) else json.dumps(v)

        # spectra
        gs = f.create_group("spectrum")
        for tag in ("raw","lsf"):
            gt = gs.create_group(tag)
            gt.create_dataset("lambda_A", data=np.asarray(spec[tag]["lambda_A"]))
            gt.create_dataset("flux",     data=np.asarray(spec[tag]["flux"]))
            gt.create_dataset("tau",      data=np.asarray(spec[tag]["tau"]))

        # small ray dump
        ad = ray.all_data()
        gray = f.create_group("ray")
        for fld, name, unit in [
            (("gas","H_p0_number_density"), "nHI_cm3",    "cm**-3"),
            (("gas","C_p1_number_density"), "nCII_cm3",   "cm**-3"),
            (("gas","Si_p2_number_density"),"nSiIII_cm3", "cm**-3"),
            (("gas","density"),              "rho_cgs",    "g/cm**3"),
            (("gas","temperature"),          "T_K",        "K"),
            (("gas","x"),                    "x_code",     "code_length"),
            (("gas","y"),                    "y_code",     "code_length"),
            (("gas","z"),                    "z_code",     "code_length"),
        ]:
            try:
                arr = ad[fld].to(unit).value
                gray.create_dataset(name, data=np.asarray(arr))
            except Exception:
                pass
        # also store the dl used for columns:
        try:
            dl_cm, kind = get_dl_cm(ray, meta["start_ckpch"], meta["end_ckpch"], verbose=False)
            gray.create_dataset("dl_cm", data=np.asarray(dl_cm))
            gray.attrs["dl_kind"] = kind
        except Exception:
            pass
    print("[SAVED]", out_path)


# -----------------------
# Batch runner
# -----------------------
def main():
    # Prep dirs
    ensure_dir(JOB_ROOT)
    ensure_dir(RAYS_DIR)
    ensure_dir(LOGS_DIR)

    # Load dataset once
    print("[INFO] Loading dataset:", CUTOUT_H5)
    ds = yt.load(CUTOUT_H5)
    print(ds)

    add_ions(ds)

    # Read rays CSV
    df = pd.read_csv(RAYS_CSV)
    print(f"[INFO] Loaded rays CSV: {RAYS_CSV}  (rows={len(df)})")
    if FILTER_MODE is not None:
        df = df[df["mode"].str.lower() == FILTER_MODE.lower()]
        print(f"[INFO] Filtered by mode='{FILTER_MODE}'  -> rows={len(df)}")
    if df.empty:
        raise RuntimeError("No rows to process after filtering.")

    # Process
    summary_rows = []
    errors = 0

    for i, row in df.iterrows():
        try:
            sid   = int(row.get("SubhaloID", SID))
            mode  = str(row.get("mode", "unknown"))
            alpha = float(row.get("alpha_deg", np.nan))

            p0 = np.array([row["p0_X_ckpch_abs"], row["p0_Y_ckpch_abs"], row["p0_Z_ckpch_abs"]], float)
            p1 = np.array([row["p1_X_ckpch_abs"], row["p1_Y_ckpch_abs"], row["p1_Z_ckpch_abs"]], float)

            rho_kpc  = float(row.get("rho_kpc", np.nan))
            phi_deg  = float(row.get("phi_deg", np.nan))
            inc_deg  = float(row.get("inc_deg", np.nan))
            rvir_kpc = float(row.get("Rvir_kpc", np.nan))

            alpha_tag = f"{int(round(alpha))}" if np.isfinite(alpha) else "NA"
            tag = f"sid{sid}_{mode}_alpha{alpha_tag}"

            # Per-ray output folder
            out_dir = os.path.join(RAYS_DIR, f"mode={mode}", f"alpha={alpha_tag}")
            ensure_dir(out_dir)

            print(f"\n[{i+1}/{len(df)}] Processing {tag} ...")

            # Build ray
            ray = make_ray(ds, p0, p1)

            # Columns
            cols = compute_columns(ray, p0, p1)

            # Spectrum
            spec = build_spectrum(ray, LINES, instr=INSTR)

            # Save figures
            save_spectrum_pngs(spec, out_dir, tag=tag)

            # Save bundle
            meta_save = dict(
                SubhaloID=sid, mode=mode, alpha_deg=alpha,
                rho_kpc=rho_kpc, phi_deg=phi_deg, inc_deg=inc_deg, Rvir_kpc=rvir_kpc,
                start_ckpch=p0.tolist(), end_ckpch=p1.tolist(),
                lines=LINES, instrument=INSTR
            )
            bundle_path = os.path.join(out_dir, f"{tag}_bundle.h5")
            save_bundle_hdf5(bundle_path, meta_save, ray, spec, cols)

            # Per-ray summary row
            srow = dict(
                SubhaloID=sid, mode=mode, alpha_deg=alpha,
                rho_kpc=rho_kpc, phi_deg=phi_deg, inc_deg=inc_deg, Rvir_kpc=rvir_kpc,
                out_dir=out_dir, bundle_h5=bundle_path,
                dl_kind=cols.get("_dl_kind"), sum_dl_kpc=cols.get("_sum_dl_kpc"),
                N_HI_cm2=cols.get("N_HI_cm2"), N_CII_cm2=cols.get("N_CII_cm2"), N_SiIII_cm2=cols.get("N_SiIII_cm2")
            )
            summary_rows.append(srow)

        except Exception as e:
            errors += 1
            msg = f"[ERROR] row {i} (mode={row.get('mode')}, alpha={row.get('alpha_deg')}): {e}"
            print(msg)
            with open(os.path.join(LOGS_DIR, "errors.txt"), "a") as f:
                f.write(msg + "\n")
            # continue to next row

    # Save master summary
    if summary_rows:
        pd.DataFrame(summary_rows).to_csv(MASTER_CSV, index=False)
        print(f"\n[OK] Wrote master summary: {MASTER_CSV}  (rows={len(summary_rows)}, errors={errors})")
    else:
        print("\n[WARN] No successful rays to summarize.", f"errors={errors}")

    print("\n[DONE] Batch finished.")
    print("      All outputs under:", JOB_ROOT)

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        print("[FATAL]", e)
        traceback.print_exc()
        raise

yt : [INFO     ] 2025-10-15 20:59:50,195 Calculating time from 1.000e+00 to be 4.356e+17 seconds
yt : [INFO     ] 2025-10-15 20:59:50,214 Parameters: current_time              = 4.355810528213311e+17 s
yt : [INFO     ] 2025-10-15 20:59:50,214 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2025-10-15 20:59:50,214 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2025-10-15 20:59:50,215 Parameters: domain_right_edge         = [35000. 35000. 35000.]
yt : [INFO     ] 2025-10-15 20:59:50,215 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2025-10-15 20:59:50,215 Parameters: current_redshift          = 2.220446049250313e-16
yt : [INFO     ] 2025-10-15 20:59:50,216 Parameters: omega_lambda              = 0.6911
yt : [INFO     ] 2025-10-15 20:59:50,216 Parameters: omega_matter              = 0.3089
yt : [INFO     ] 2025-10-15 20:59:50,216 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2025-10-15 20:59:50,216 Parameters: hubble_con

[INFO] Loading dataset: ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/cutout_ALLFIELDS_sphere_2p1Rvir_sub563732.hdf5
cutout_ALLFIELDS_sphere_2p1Rvir_sub563732


Loading particle index: 100%|██████████| 13/13 [00:00<00:00, 8951.89it/s]
yt : [WARNING  ] 2025-10-15 20:59:50,457 Field ('gas', 'H_p0_number_density') already exists. Not clobbering.
yt : [WARNING  ] 2025-10-15 20:59:50,457 Field ('gas', 'H_p0_density') already exists. Not clobbering.
yt : [INFO     ] 2025-10-15 20:59:50,473 Getting segment at z = 2.220446049250313e-16: [0.66584318 0.51218094 0.24018352] unitary to [0.66763032 0.51957756 0.25220259] unitary.
yt : [INFO     ] 2025-10-15 20:59:50,474 Getting subsegment: [0.66584318 0.51218094 0.24018352] unitary to [0.66763032 0.51957756 0.25220259] unitary.


[INFO] Ion fields added (or already present): ['H I', 'C II', 'Si III']
[INFO] Loaded rays CSV: rays_and_recipes_sid563732_snap99/rays_sid563732.csv  (rows=720)

[1/720] Processing sid563732_noflip_alpha0 ...
[RAY] Start (code_length): [23304.51112799 17926.33292137  8406.42318879] code_length
[RAY] End   (code_length): [23367.06130566 18185.21468047  8827.09071356] code_length
[RAY] start inside domain: True
[RAY] end inside domain: True


yt : [INFO     ] 2025-10-15 20:59:51,325 Saving field data to yt dataset: ray.h5.
yt : [INFO     ] 2025-10-15 20:59:51,338 Parameters: current_time              = 4.355810528213311e+17 s
yt : [INFO     ] 2025-10-15 20:59:51,338 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2025-10-15 20:59:51,338 Parameters: domain_left_edge          = [0. 0. 0.] code_length
yt : [INFO     ] 2025-10-15 20:59:51,338 Parameters: domain_right_edge         = [35000. 35000. 35000.] code_length
yt : [INFO     ] 2025-10-15 20:59:51,339 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2025-10-15 20:59:51,339 Parameters: current_redshift          = 2.220446049250313e-16
yt : [INFO     ] 2025-10-15 20:59:51,339 Parameters: omega_lambda              = 0.6911
yt : [INFO     ] 2025-10-15 20:59:51,339 Parameters: omega_matter              = 0.3089
yt : [INFO     ] 2025-10-15 20:59:51,340 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2025-10-15 20:59:51,340 Parameters: h

...

[SAVED] ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/rays_and_spectra_sid563732_snap99/rays/mode=noflip/alpha=359/sid563732_noflip_alpha359_spectrum_zooms.png
[SAVED] ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/rays_and_spectra_sid563732_snap99/rays/mode=noflip/alpha=359/sid563732_noflip_alpha359_bundle.h5

[720/720] Processing sid563732_flip_alpha359 ...
[RAY] Start (code_length): [23283.61509456 18016.63309578  8968.66994782] code_length
[RAY] End   (code_length): [23463.63974087 17823.81404941  8546.40770892] code_length
[RAY] start inside domain: True
[RAY] end inside domain: True


yt : [INFO     ] 2025-10-15 21:13:33,939 Saving field data to yt dataset: ray.h5.
yt : [INFO     ] 2025-10-15 21:13:33,951 Parameters: current_time              = 4.355810528213311e+17 s
yt : [INFO     ] 2025-10-15 21:13:33,951 Parameters: domain_dimensions         = [1 1 1]
yt : [INFO     ] 2025-10-15 21:13:33,952 Parameters: domain_left_edge          = [0. 0. 0.] code_length
yt : [INFO     ] 2025-10-15 21:13:33,952 Parameters: domain_right_edge         = [35000. 35000. 35000.] code_length
yt : [INFO     ] 2025-10-15 21:13:33,952 Parameters: cosmological_simulation   = True
yt : [INFO     ] 2025-10-15 21:13:33,952 Parameters: current_redshift          = 2.220446049250313e-16
yt : [INFO     ] 2025-10-15 21:13:33,953 Parameters: omega_lambda              = 0.6911
yt : [INFO     ] 2025-10-15 21:13:33,953 Parameters: omega_matter              = 0.3089
yt : [INFO     ] 2025-10-15 21:13:33,953 Parameters: omega_radiation           = 0.0
yt : [INFO     ] 2025-10-15 21:13:33,953 Parameters: h

[RAY] N segments: 409 | geometric length ≈ 497.889 ckpc/h
[DL] Using native field ('gas', 'dl') (min=6.006e+11 cm, max=4.686e+22 cm)
=== Integrated columns (cm^-2) ===
N_HI_cm2  : 3.360588e+13   (logN = 13.526)
N_CII_cm2 : 7.663227e+07   (logN = 7.884)
N_SiIII_cm2: 1.414365e+07   (logN = 7.151)
[DL] kind=native | Σdl ≈ 737.623 kpc


Adding line - Si III 1206 [1206.500000 A]: 100%|██████████| 409/409 [00:00<00:00, 25425.68it/s]
yt : [INFO     ] 2025-10-15 21:13:34,122 Not adding continuum Ly C: insufficient column density or out of range


[SPEC] λ grid: 1150.00–1450.00 Å  (N=30001)  |  instrument=COS-G130M
[SAVED] ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/rays_and_spectra_sid563732_snap99/rays/mode=flip/alpha=359/sid563732_flip_alpha359_spectrum_full.png
[SAVED] ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/rays_and_spectra_sid563732_snap99/rays/mode=flip/alpha=359/sid563732_flip_alpha359_spectrum_zooms.png
[SAVED] ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/rays_and_spectra_sid563732_snap99/rays/mode=flip/alpha=359/sid563732_flip_alpha359_bundle.h5

[OK] Wrote master summary: ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/rays_and_spectra_sid563732_snap99/summary_all_rays.csv  (rows=720, errors=0)

[DONE] Batch finished.
      All outputs under: ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/rays_and_spectra_sid563732_snap99


### Analyzing the results of the spectra

In [10]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Analyze batch ray+spectra results.

Inputs (created by the batch ray builder):
  JOB_ROOT/
    summary_all_rays.csv
    rays/mode=.../alpha=.../*_bundle.h5

Outputs here:
  JOB_ROOT/
    analysis/
      augmented_summary.csv
      figs/*.png
      report_analysis.pdf
"""

import os
import json
import math
import h5py
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# -----------------------------
# CONFIG — edit if you need to
# -----------------------------
SID  = 563732
SNAP = 99

OUTPUT_BASE = "../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732"
JOB_ROOT    = os.path.join(OUTPUT_BASE, f"rays_and_spectra_sid{SID}_snap{SNAP}")
MASTER_CSV  = os.path.join(JOB_ROOT, "summary_all_rays.csv")

ANALYSIS_DIR = os.path.join(JOB_ROOT, "analysis")
FIG_DIR      = os.path.join(ANALYSIS_DIR, "figs")
REPORT_PDF   = os.path.join(ANALYSIS_DIR, "report_analysis.pdf")

# Nice defaults for plots
mpl.rcParams.update({
    "savefig.dpi": 200,
    "figure.dpi": 130,
    "font.size": 11,
    "axes.grid": True,
    "grid.alpha": 0.25,
    "axes.linewidth": 1.0,
    "xtick.direction": "in",
    "ytick.direction": "in",
    "xtick.minor.visible": True,
    "ytick.minor.visible": True,
})

COLORS = {
    "all":   "#7f8c8d",
    "noflip":"#1f77b4",
    "flip":  "#d62728",
}

# -----------------------------
# Helpers
# -----------------------------
def ensure_dir(p):
    os.makedirs(p, exist_ok=True)
    return p

def _log10_pos(x):
    x = np.asarray(x, float)
    m = (x > 0) & np.isfinite(x)
    out = np.full_like(x, np.nan, dtype=float)
    out[m] = np.log10(x[m])
    return out

def _safe_len_from_bundle(h5_path):
    """
    Return (n_cells, dl_kind, sum_dl_kpc) from a bundle if available.
    """
    try:
        with h5py.File(h5_path, "r") as f:
            rayg = f.get("ray", None)
            if rayg is None:
                return (np.nan, None, np.nan)
            # Prefer explicit dl array length
            if "dl_cm" in rayg:
                dl = np.array(rayg["dl_cm"][:], dtype=float)
                n = dl.size
                sum_kpc = float(dl.sum() / 3.08567758e21)
                kind = rayg.attrs.get("dl_kind", None)
                if isinstance(kind, bytes):
                    kind = kind.decode("utf-8", "ignore")
                return (n, kind, sum_kpc)
            # Fallback: any per-cell array length
            for key in ("nHI_cm3", "nCII_cm3", "nSiIII_cm3", "rho_cgs", "T_K", "x_code"):
                if key in rayg:
                    n = int(np.array(rayg[key][:]).size)
                    return (n, None, np.nan)
    except Exception:
        pass
    return (np.nan, None, np.nan)

def _norm_rho(df):
    if {"rho_kpc","Rvir_kpc"}.issubset(df.columns):
        rv = df["Rvir_kpc"].replace(0, np.nan).astype(float)
        return df["rho_kpc"].astype(float) / rv
    return pd.Series(np.nan, index=df.index)

# -----------------------------
# Load & augment
# -----------------------------
def load_and_augment(master_csv, peek_bundles=True):
    if not os.path.isfile(master_csv):
        raise FileNotFoundError(f"Missing {master_csv}")

    df = pd.read_csv(master_csv)
    print(f"[INFO] Loaded master summary: {master_csv} (rows={len(df)})")

    # Compute logs of columns
    for c_src, c_log in [
        ("N_HI_cm2", "logN_HI"),
        ("N_CII_cm2","logN_CII"),
        ("N_SiIII_cm2","logN_SiIII"),
    ]:
        if c_src in df.columns:
            df[c_log] = _log10_pos(df[c_src].values)
        else:
            df[c_log] = np.nan

    # Normalize impact parameter
    df["rho_over_Rvir"] = _norm_rho(df)

    # Round alpha to integer for grouping
    if "alpha_deg" in df.columns:
        df["alpha_int"] = np.floor(df["alpha_deg"].astype(float) + 0.5).astype("Int64")
    else:
        df["alpha_int"] = pd.Series([pd.NA] * len(df), dtype="Int64")

    # Peek into bundles for n_cells and (if missing) sum_dl_kpc/dl_kind
    n_cells_list, dl_kind_list, sum_dl_kpc_list = [], [], []
    if peek_bundles and "bundle_h5" in df.columns:
        print("[INFO] Scanning bundles for n_cells / dl …")
        for p in df["bundle_h5"].fillna(""):
            if isinstance(p, str) and os.path.isfile(p):
                n, kind, sum_kpc = _safe_len_from_bundle(p)
            else:
                n, kind, sum_kpc = (np.nan, None, np.nan)
            n_cells_list.append(n)
            dl_kind_list.append(kind)
            sum_dl_kpc_list.append(sum_kpc)
    else:
        n_cells_list = [np.nan]*len(df)
        dl_kind_list = [None]*len(df)
        sum_dl_kpc_list = [np.nan]*len(df)

    # Prefer values already in master; fill NaNs with bundle-derived
    if "sum_dl_kpc" in df.columns:
        s = df["sum_dl_kpc"].astype(float)
        fill = pd.Series(sum_dl_kpc_list)
        df["sum_dl_kpc"] = np.where(np.isfinite(s), s, fill)
    else:
        df["sum_dl_kpc"] = np.array(sum_dl_kpc_list, float)

    if "dl_kind" in df.columns:
        df["dl_kind"] = df["dl_kind"].astype("string")
        df["dl_kind"] = df["dl_kind"].fillna(pd.Series(dl_kind_list, dtype="string"))
    else:
        df["dl_kind"] = pd.Series(dl_kind_list, dtype="string")

    df["n_cells"] = np.array(n_cells_list, float)

    # Save augmented table
    ensure_dir(ANALYSIS_DIR)
    aug_csv = os.path.join(ANALYSIS_DIR, "augmented_summary.csv")
    df.to_csv(aug_csv, index=False)
    print(f"[OK] Wrote {aug_csv}")
    return df

# -----------------------------
# Plotters
# -----------------------------
def _finalize(fig, ax, title, out_png, pdf=None):
    ax.set_title(title)
    fig.tight_layout()
    fig.savefig(out_png)
    if pdf is not None:
        pdf.savefig(fig)
    plt.close(fig)
    print("[saved]", out_png)

def hist_columns(df, out_dir, pdf=None):
    ensure_dir(out_dir)
    cols = [("logN_HI","log10 N(H I) [cm⁻²]"),
            ("logN_CII","log10 N(C II) [cm⁻²]"),
            ("logN_SiIII","log10 N(Si III) [cm⁻²]")]
    for key, label in cols:
        if key not in df.columns: 
            continue
        fig, ax = plt.subplots(figsize=(6.0,4.0))
        m = np.isfinite(df[key].values)
        ax.hist(df.loc[m, key], bins=30, color=COLORS["all"], alpha=0.6, label="all")
        # Per-mode overlays if present
        if "mode" in df.columns:
            for mode, col in [("noflip", COLORS["noflip"]), ("flip", COLORS["flip"])]:
                mm = m & (df["mode"].astype(str).str.lower() == mode)
                if mm.any():
                    ax.hist(df.loc[mm, key], bins=30, histtype="step", color=col, lw=1.6, label=mode)
        ax.set_xlabel(label); ax.set_ylabel("Count"); ax.legend()
        _finalize(fig, ax, f"Histogram of {label}", os.path.join(out_dir, f"hist_{key}.png"), pdf)

def hist_pathlengths_cells(df, out_dir, pdf=None):
    ensure_dir(out_dir)
    # Σdl
    if "sum_dl_kpc" in df.columns:
        fig, ax = plt.subplots(figsize=(6.0,4.0))
        m = np.isfinite(df["sum_dl_kpc"].values)
        ax.hist(df.loc[m, "sum_dl_kpc"], bins=30, color="#16a085", alpha=0.8)
        ax.set_xlabel("Σ dl  [kpc]"); ax.set_ylabel("Count")
        _finalize(fig, ax, "Histogram of total path length", os.path.join(out_dir, "hist_sum_dl_kpc.png"), pdf)
    # n_cells
    if "n_cells" in df.columns:
        fig, ax = plt.subplots(figsize=(6.0,4.0))
        m = np.isfinite(df["n_cells"].values)
        ax.hist(df.loc[m, "n_cells"], bins=30, color="#8e44ad", alpha=0.8)
        ax.set_xlabel("Number of intersected cells"); ax.set_ylabel("Count")
        _finalize(fig, ax, "Histogram of n_cells", os.path.join(out_dir, "hist_n_cells.png"), pdf)

def bar_dl_kind(df, out_dir, pdf=None):
    if "dl_kind" not in df.columns: 
        return
    ensure_dir(out_dir)
    fig, ax = plt.subplots(figsize=(6.0,4.0))
    counts = df["dl_kind"].fillna("unknown").value_counts()
    ax.bar(counts.index, counts.values, color="#34495e")
    for i, v in enumerate(counts.values):
        ax.text(i, v, str(v), ha="center", va="bottom")
    ax.set_xlabel("dl_kind"); ax.set_ylabel("Count")
    _finalize(fig, ax, "Counts of dl_kind (native vs approx)", os.path.join(out_dir, "bar_dl_kind.png"), pdf)

def scatter_vs_alpha(df, out_dir, pdf=None):
    if "alpha_deg" not in df.columns:
        return
    ensure_dir(out_dir)
    for key, label in [("logN_HI","log10 N(H I)"), ("logN_CII","log10 N(C II)"), ("logN_SiIII","log10 N(Si III)")]:
        if key not in df.columns: 
            continue
        fig, ax = plt.subplots(figsize=(6.6,4.2))
        m = np.isfinite(df[key].values) & np.isfinite(df["alpha_deg"].values)
        # jitter alpha a bit for visibility
        a = df.loc[m, "alpha_deg"].values
        y = df.loc[m, key].values
        if "mode" in df.columns:
            modes = df.loc[m, "mode"].astype(str).str.lower().values
            for mode, col in [("noflip", COLORS["noflip"]), ("flip", COLORS["flip"])]:
                mm = (modes == mode)
                ax.scatter(a[mm], y[mm], s=16, color=col, alpha=0.7, label=mode)
        else:
            ax.scatter(a, y, s=16, color=COLORS["all"], alpha=0.7, label="all")
        ax.set_xlabel("alpha [deg]"); ax.set_ylabel(label)
        ax.legend()
        _finalize(fig, ax, f"{label} vs alpha", os.path.join(out_dir, f"scatter_{key}_vs_alpha.png"), pdf)

def scatter_vs_rhooverrvir(df, out_dir, pdf=None):
    if "rho_over_Rvir" not in df.columns:
        return
    ensure_dir(out_dir)
    for key, label in [("logN_HI","log10 N(H I)"), ("logN_CII","log10 N(C II)"), ("logN_SiIII","log10 N(Si III)")]:
        if key not in df.columns: 
            continue
        fig, ax = plt.subplots(figsize=(6.6,4.2))
        m = np.isfinite(df[key].values) & np.isfinite(df["rho_over_Rvir"].values)
        x = df.loc[m, "rho_over_Rvir"].values
        y = df.loc[m, key].values
        if "mode" in df.columns:
            modes = df.loc[m, "mode"].astype(str).str.lower().values
            for mode, col in [("noflip", COLORS["noflip"]), ("flip", COLORS["flip"])]:
                mm = (modes == mode)
                ax.scatter(x[mm], y[mm], s=16, color=col, alpha=0.7, label=mode)
        else:
            ax.scatter(x, y, s=16, color=COLORS["all"], alpha=0.7, label="all")
        ax.set_xlabel("rho / Rvir"); ax.set_ylabel(label)
        ax.axvline(1.0, ls="--", color="k", lw=0.8, alpha=0.5)
        ax.legend()
        _finalize(fig, ax, f"{label} vs rho/Rvir", os.path.join(out_dir, f"scatter_{key}_vs_rhoRvir.png"), pdf)

def correlation_heatmap(df, out_dir, pdf=None):
    ensure_dir(out_dir)
    # pick numeric columns
    cand = ["logN_HI","logN_CII","logN_SiIII","sum_dl_kpc","n_cells","rho_kpc","Rvir_kpc","rho_over_Rvir","alpha_deg"]
    X = df[[c for c in cand if c in df.columns]].copy()
    X = X.apply(pd.to_numeric, errors="coerce")
    C = X.corr(method="pearson", numeric_only=True)
    fig, ax = plt.subplots(figsize=(7.8,6.2))
    im = ax.imshow(C.values, cmap="coolwarm", vmin=-1, vmax=1)
    ax.set_xticks(np.arange(C.shape[1])); ax.set_xticklabels(C.columns, rotation=45, ha="right")
    ax.set_yticks(np.arange(C.shape[0])); ax.set_yticklabels(C.index)
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            v = C.values[i,j]
            if np.isfinite(v):
                ax.text(j, i, f"{v:+.2f}", ha="center", va="center", color="black")
    fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04, label="Pearson r")
    _finalize(fig, ax, "Correlation heatmap", os.path.join(out_dir, "corr_heatmap.png"), pdf)

def cdf_per_mode(df, out_dir, pdf=None):
    ensure_dir(out_dir)
    for key, label in [("logN_HI","log10 N(H I)"), ("logN_CII","log10 N(C II)"), ("logN_SiIII","log10 N(Si III)")]:
        if key not in df.columns: 
            continue
        fig, ax = plt.subplots(figsize=(6.6,4.2))
        for mode, col in [("noflip", COLORS["noflip"]), ("flip", COLORS["flip"])]:
            m = (df["mode"].astype(str).str.lower() == mode) & np.isfinite(df[key].values)
            if not m.any(): 
                continue
            x = np.sort(df.loc[m, key].values)
            y = np.linspace(0, 1, x.size, endpoint=True)
            ax.plot(x, y, color=col, lw=2.0, label=mode)
        ax.set_xlabel(label); ax.set_ylabel("CDF")
        ax.legend()
        _finalize(fig, ax, f"CDF of {label} (by mode)", os.path.join(out_dir, f"cdf_{key}_by_mode.png"), pdf)

def print_quick_stats(df):
    print("\n=== Quick stats ===")
    for key in ["logN_HI","logN_CII","logN_SiIII","sum_dl_kpc","n_cells","rho_over_Rvir"]:
        if key in df.columns:
            s = pd.Series(df[key], dtype=float)
            s = s.replace([np.inf,-np.inf], np.nan).dropna()
            if s.empty:
                continue
            q = s.quantile([0.1,0.5,0.9])
            print(f"{key:>14s}: N={s.size:4d}  mean={s.mean():.3f}  std={s.std():.3f}  "
                  f"p10={q.iloc[0]:.3f}  p50={q.iloc[1]:.3f}  p90={q.iloc[2]:.3f}")
    if "dl_kind" in df.columns:
        print("\ndl_kind counts:")
        print(df["dl_kind"].fillna("unknown").value_counts())

# -----------------------------
# Main
# -----------------------------
def main():
    ensure_dir(ANALYSIS_DIR); ensure_dir(FIG_DIR)

    df = load_and_augment(MASTER_CSV, peek_bundles=True)

    # Build a multi-page PDF report and individual PNGs
    with PdfPages(REPORT_PDF) as pdf:
        hist_columns(df, FIG_DIR, pdf=pdf)
        hist_pathlengths_cells(df, FIG_DIR, pdf=pdf)
        bar_dl_kind(df, FIG_DIR, pdf=pdf)
        scatter_vs_alpha(df, FIG_DIR, pdf=pdf)
        scatter_vs_rhooverrvir(df, FIG_DIR, pdf=pdf)
        correlation_heatmap(df, FIG_DIR, pdf=pdf)
        if "mode" in df.columns:
            cdf_per_mode(df, FIG_DIR, pdf=pdf)

    print_quick_stats(df)
    print(f"\n[OK] Wrote report: {REPORT_PDF}")
    print(f"[OK] PNGs in: {FIG_DIR}")
    print(f"[OK] Augmented CSV in: {os.path.join(ANALYSIS_DIR, 'augmented_summary.csv')}")

if __name__ == "__main__":
    main()

[INFO] Loaded master summary: ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/rays_and_spectra_sid563732_snap99/summary_all_rays.csv (rows=720)
[INFO] Scanning bundles for n_cells / dl …
[OK] Wrote ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/rays_and_spectra_sid563732_snap99/analysis/augmented_summary.csv
[saved] ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/rays_and_spectra_sid563732_snap99/analysis/figs/hist_logN_HI.png
[saved] ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/rays_and_spectra_sid563732_snap99/analysis/figs/hist_logN_CII.png
[saved] ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/rays_and_spectra_sid563732_snap99/analysis/figs/hist_logN_SiIII.png
[saved] ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos/sub_563732/rays_and_spectra_sid563732_snap99/analysis/figs/hist_sum_dl_kpc.png
[saved] ../../../../ASU Dropbox/Tanmay Singh/COS_GASS/TNG_Subhalos